In [ ]:
import sqlite3
import pandas as pd

# 1. BAĞLANTI VE AYARLAR
connection = sqlite3.connect("ticaret_sistemi.db")
cursor = connection.cursor()
cursor.execute("PRAGMA foreign_keys = ON;") # İlişkisel bütünlüğü aktif et

# ---------------------------------------------------------
# 2. TABLO OLUŞTURMA (CREATE)
# ---------------------------------------------------------
# Önce bağımlı tabloları (çocukları) siliyoruz, sonra ana tabloları (babaları)
tablolar = ["parca", "siparis", "personel", "musteri"]
for tablo in tablolar:
    cursor.execute(f"DROP TABLE IF EXISTS {tablo}")

# 1. Müşteri Tablosu
cursor.execute("""
CREATE TABLE musteri (
    musterino INTEGER PRIMARY KEY,
    adi TEXT,
    soyadi TEXT,
    istelefon TEXT,
    ceptelefon TEXT
);
""")

# 2. Personel Tablosu
cursor.execute("""
CREATE TABLE personel (
    perno INTEGER PRIMARY KEY,
    adi TEXT,
    soyadi TEXT,
    giris_tarih TEXT,
    maasi REAL
);
""")

# 3. Sipariş Tablosu (Müşteri ve Personele bağlı)
cursor.execute("""
CREATE TABLE siparis (
    siparisno INTEGER PRIMARY KEY,
    musno INTEGER,
    siparistarih TEXT,
    elemanno INTEGER,
    toplam REAL,
    FOREIGN KEY(musno) REFERENCES musteri(musterino),
    FOREIGN KEY(elemanno) REFERENCES personel(perno)
);
""")

# 4. Parça Tablosu (Siparişe bağlı)
# Not: 'sirano' tablonun kendi ID'si, 'parcano' ürünün kodu gibi düşünüldü.
cursor.execute("""
CREATE TABLE parca (
    sirano INTEGER PRIMARY KEY,
    sipno INTEGER,
    parcano INTEGER,
    adi TEXT,
    miktar INTEGER,
    FOREIGN KEY(sipno) REFERENCES siparis(siparisno)
);
""")

print("Tablolar başarıyla oluşturuldu.")

# ---------------------------------------------------------
# 3. VERİ EKLEME (INSERT)
# ---------------------------------------------------------

# Müşteriler
cursor.executemany("INSERT INTO musteri VALUES (?,?,?,?,?)", [
    (1, 'Ahmet', 'Yılmaz', '0212-111-1111', '0555-111-2233'),
    (2, 'Ayşe', 'Demir', '0216-222-2222', '0532-444-5566')
])

# Personeller
cursor.executemany("INSERT INTO personel VALUES (?,?,?,?,?)", [
    (10, 'Mehmet', 'Satış', '2020-01-15', 15000),
    (11, 'Zeynep', 'Kasa', '2021-05-20', 17500)
])

# Siparişler (Hangi müşteri, hangi personelden aldı?)
cursor.executemany("INSERT INTO siparis VALUES (?,?,?,?,?)", [
    (1001, 1, '2023-10-01', 10, 5000.0),  # Ahmet, Mehmet'ten 5000 TL'lik ürün aldı
    (1002, 2, '2023-10-02', 11, 2500.0),  # Ayşe, Zeynep'ten 2500 TL'lik ürün aldı
    (1003, 1, '2023-10-05', 10, 1000.0)   # Ahmet, tekrar Mehmet'ten aldı
])

# Parçalar (Siparişin detayları: Hangi siparişte hangi ürünler var?)
cursor.executemany("INSERT INTO parca VALUES (?,?,?,?,?)", [
    (1, 1001, 50, 'Laptop', 1),       # 1001 nolu siparişte 1 Laptop var
    (2, 1001, 51, 'Mouse', 1),        # 1001 nolu siparişte 1 Mouse var
    (3, 1002, 60, 'Monitör', 2),      # 1002 nolu siparişte 2 Monitör var
    (4, 1003, 70, 'Klavye', 1)        # 1003 nolu siparişte 1 Klavye var
])

connection.commit()
print("Veriler başarıyla eklendi.")

# ---------------------------------------------------------
# 4. KONTROL SORGUSU (RAPOR)
# ---------------------------------------------------------
# Müşteri adı, Sipariş Tarihi, Ürün Adı ve Personel Adını birleştiren rapor
sql_rapor = """
SELECT 
    m.adi || ' ' || m.soyadi as Müşteri,
    s.siparistarih,
    p.adi as Ürün,
    p.miktar,
    per.adi as Satış_Personeli
FROM parca p
JOIN siparis s ON p.sipno = s.siparisno
JOIN musteri m ON s.musno = m.musterino
JOIN personel per ON s.elemanno = per.perno;
"""

print("-" * 50)
print("SİPARİŞ DETAY RAPORU:")
df = pd.read_sql(sql_rapor, connection)
print(df)

def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None



Tablolar başarıyla oluşturuldu.
Veriler başarıyla eklendi.
--------------------------------------------------
SİPARİŞ DETAY RAPORU:
        Müşteri siparistarih     Ürün  miktar Satış_Personeli
0  Ahmet Yılmaz   2023-10-01   Laptop       1          Mehmet
1  Ahmet Yılmaz   2023-10-01    Mouse       1          Mehmet
2    Ayşe Demir   2023-10-02  Monitör       2          Zeynep
3  Ahmet Yılmaz   2023-10-05   Klavye       1          Mehmet


In [ ]:
#en fazla sipariş veren müşterinin aldığı parçaların toplam miktarını bulunuz

In [ ]:
#müşteri listesinden hiç sipariş vermemiş müşterileri listeleyiniz v1

In [ ]:
#müşteri listesinden hiç sipariş vermemiş müşterileri listeleyiniz v2

In [ ]:
#ortalama maaştan fazla maaş olan personel en çok hangi parçayı satmıştır

In [ ]:
connection.close()